In [ ]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 
import pickle
import warnings

from utils import *

from sensor_params import *

from models.training import *

from models.multi_step.unet1D_regressor import *
from models.multi_step.unet1D_regressor import get_model as get_unet1D

# from models.multi_step.unet1D_DIL_regressor import *
# from models.multi_step.unet1D_DIL_regressor import get_model as get_DIL_unet1D

from models.multi_step.LSTMVanilla import *
from models.multi_step.LSTMVanilla import get_model as get_LSTM

# from models.multi_step.unet1D_LSTM_regressor import *
# from models.multi_step.unet1D_LSTM_regressor import get_model as get_unet1DLSTM

# from models.multi_step.unet1D_nonCompres_regressor import *
# from models.multi_step.unet1D_nonCompres_regressor import get_model as get_unet1D_nonCompres

from evaluation.multi_step import *


# Dataset path 
DATABASE_PATH = r"C:\Users\aralmeida\OneDrive - Universidad de Las Palmas de Gran Canaria\Doctorado\Bases de datos\WARIFA\Mikael T1DM"

filename = "MIKAEL_data.json"

parent_directory = r"C:\Users\aralmeida\Downloads"
experiments_folder = r"\T1DM_pred_experiments" 


In [ ]:
N = 144
tau = 1
PH = 30
pred_steps = 6
name = 'LSTM36-himar-rep'

In [ ]:
with open(DATABASE_PATH+'\CGM.pk1', 'rb') as f:
    sgv_data_dict = pickle.load(f)
os.chdir(parent_directory)

In [ ]:
X, Y, X_times, Y_times = get_CGM_X_Y_multistep(sgv_data_dict, sensor_Mikael, N, tau, PH, experiments_folder, plot=False, verbose = 0)

In [ ]:
X_train = X[np.where(X_times[:,N-1] <= pd.to_datetime('2021-05-31 00:00:00'))[0]]
Y_train = Y[np.where((Y_times[:,0] < pd.to_datetime('2021-05-30 23:59:59')))[0]]
X_test = X[np.where((X_times[:,N-1] > pd.to_datetime('2021-06-01 00:00:00')))[0]]
Y_test = Y[np.where((Y_times[:,0] > pd.to_datetime('2021-06-01 00:00:00')))[0]]

In [ ]:
os.chdir(r"C:\Users\aralmeida\Downloads\T1DM_pred_experiments\N144_stride1_step6\training")

In [ ]:
# Model prediction
model_trained = tf.keras.models.load_model(name+'.h5')
Y_pred = model_trained.predict(X_test)

In [ ]:
model_evaluation(name, X_test, Y_test, pred_steps) 

In [15]:
a = np.array([5,9,78,9,4])
a_norm = (a - np.min(a))/(np.max(a) - np.min(a))
print(a_norm)
a_denorm = a_norm*(np.max(a) - np.min(a)) + np.min(a)
print(a_denorm)

[0.01351351 0.06756757 1.         0.06756757 0.        ]
[ 5.  9. 78.  9.  4.]


In [16]:
pred_norm = np.array([0.01351351, 0.06756757, 1, 0.06756757, 0])
pred_denorm = pred_norm*(np.max(a) - np.min(a)) + np.min(a)
pred_denorm

array([ 4.99999974,  9.00000018, 78.        ,  9.00000018,  4.        ])